# Implementing the EffTox Dose-Finding Design in the Matchpoint Trials

Brock _et al_., in submission

## Dose Ambivalence

In [1]:
import numpy as np
from scipy.stats import norm

from clintrials.dosefinding.efftox import EffTox, LpNormCurve

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
real_doses = [7.5, 15, 30, 45]
trial_size = 30
cohort_size = 3
first_dose = 3
prior_tox_probs = (0.025, 0.05, 0.1, 0.25)
prior_eff_probs = (0.2, 0.3, 0.5, 0.6)
tox_cutoff = 0.40
eff_cutoff = 0.45
tox_certainty = 0.05
eff_certainty = 0.03

In [3]:
mu_t_mean, mu_t_sd = -5.4317, 2.7643
beta_t_mean, beta_t_sd = 3.1761, 2.7703
mu_e_mean, mu_e_sd = -0.8442, 1.9786
beta_e_1_mean, beta_e_1_sd = 1.9857, 1.9820
beta_e_2_mean, beta_e_2_sd = 0, 0.2
psi_mean, psi_sd = 0, 1
efftox_priors = [
    norm(loc=mu_t_mean, scale=mu_t_sd),
    norm(loc=beta_t_mean, scale=beta_t_sd),
    norm(loc=mu_e_mean, scale=mu_e_sd),
    norm(loc=beta_e_1_mean, scale=beta_e_1_sd),
    norm(loc=beta_e_2_mean, scale=beta_e_2_sd),
    norm(loc=psi_mean, scale=psi_sd),
    ]

In [4]:
hinge_points = [(0.4, 0), (1, 0.7), (0.5, 0.4)]
metric = LpNormCurve(hinge_points[0][0], hinge_points[1][1], hinge_points[2][0], hinge_points[2][1])

In [5]:
et = EffTox(real_doses, efftox_priors, tox_cutoff, eff_cutoff, tox_certainty, eff_certainty, metric, trial_size,
            first_dose)

## Dose ambivalence after 3NTE

Outcomes for a patient are represented by a three item tuple, where:

- first item is 1-based dose-index give (i.e. 3 is dose-level 3);
- second item is 1 if toxicity happened, else 0;
- third item is 1 if efficacy happened, else 0.

Outcomes for several patients are represented as lists:

outcomes = [(3, 0, 0), (3, 1, 0), (3, 1, 0),
            (3, 0, 1), (3, 0, 1), (3, 0, 0),
           ]

In [42]:
outcomes = [(3, 0, 0), (3, 1, 0), (3, 0, 1)]

In [43]:
et.reset()
np.random.seed(123)
et.update(outcomes)

3

In [44]:
et.reset()
np.random.seed(321)
et.update(outcomes)

4

We define a simple function to calculate next dose based on some outcomes:

In [45]:
def get_next_dose(trial, outcomes, **kwargs):
    trial.reset()
    next_dose = trial.update(outcomes, **kwargs)
    return next_dose

And then run that a number of times. For indication, 100 iterations will suffice. It takes a wee while...

In [46]:
np.random.seed(123)
replicates = [get_next_dose(et, outcomes1, n=10**5) for i in range(100)]

In [47]:
doses, freq = np.unique(replicates, return_counts=True)
zip(doses, 1.0 * freq / len(replicates))

[(3, 0.56000000000000005), (4, 0.44)]

In [48]:
et.tabulate()

,Dose,N,Efficacies,Toxicities,EffRate,ToxRate,P(Eff),P(Tox),P(AccEff),P(AccTox),Admissible,Utility
0,1,0,0,0,NaN,NaN,0.169569,0.041893,0.131056,0.968148,True,-0.385060
1,2,0,0,0,NaN,NaN,0.209500,0.053362,0.120474,0.974309,True,-0.319252
2,3,3,1,1,0.333333,0.333333,0.379005,0.263953,0.365033,0.782415,True,-0.095079
3,4,0,0,0,NaN,NaN,0.510385,0.570060,0.487628,0.355802,True,-0.139662


We see that all doses are admissible in this instance, and that the utilities of dose-levels 3 and 4 are very similar. _Dose Ambivalence_ is the likely result, i.e. after observing 3NTE in the Matchpoint trial, the design would have recommended dose 3 or dose 4. The reason is made plain by the plot below.

The posterior distributions of the utility of doses 3 and 4 largely occupy the same space so picking between them is difficult. In the Ambivalence.ipynb tutorial, we demonstrate a method for dealing with dose ambivalence.

The plot above is similar (but not identical) to Figure 2 in the publication. I used the R package `ggplot2` to produce the plots for the paper because the R package is more mature than the Python version. For instance, I could not get a legend to appear in Python.